## KNN

- k-近邻算法采用测量不同特征值之间的距离方法进行分类。
- 优点：精度高、对异常值不敏感、无数据输入假定
- 缺点：计算复杂度高、空间复杂度高
- 适用数据范围：数值型和标称型

存在一个训练样本集，并且每个样本都存在标签（有监督学习）。输入没有标签的新样本数据后，将新数据的每个特征与样本集中数据对应的特征进行比较，然后算法提取出与样本集中特征最相似的数据（最近邻）的分类标签。一般来说，我们只选择样本数据集中前k个最相似的数据，这就是k-近邻算法中k的出处，而且k通常不大于20。最后选择k个最相似数据中出现次数最多的分类，作为新数据的分类。

In [42]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_table

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

读取TXT文件，各个字段之间使用空格分割，为每个字段设置一个name，生成dataframe

In [43]:
df = pd.read_table('/home/pytest/MLaction/Ch02_knn/datingTestSet.txt', 
                   sep='[ |\t]', names=['miles','gameUsed','iceCream','fitness'], 
                   encoding='utf-8', engine='python')
df[0:1]

,miles,gameUsed,iceCream,fitness
0,40920,8.326976,0.953952,largeDoses


首先观察三个特征参数，对特征参数处理的目标是转化为绝对值大小相近的数值，根据这个目标，显然需要对miles和gameUsed两个字段进行标准化，将其值缩放到[-1,1]之间，变换后需要将原来的属性删除

In [44]:
import sklearn.preprocessing as preprocessing

# 使用标准缩放器
scaler = preprocessing.StandardScaler()

# 生成全局缩放参数
miles_scale_param = scaler.fit(df['miles'].values.reshape(-1,1))
# 根据缩放参数生成相应的新属性
df['miles_scaled'] = scaler.fit_transform(df['miles'].values.reshape(-1,1), miles_scale_param)

gameUsed_scale_param = scaler.fit(df['gameUsed'].values.reshape(-1,1))
df['gameUsed_scaled'] = scaler.fit_transform(df['gameUsed'].values.reshape(-1,1), gameUsed_scale_param)

# 删除原来的属性
df.drop(['miles', 'gameUsed'], axis=1, inplace=True)

df[0:1]

,iceCream,fitness,miles_scaled,gameUsed_scaled
0,0.953952,largeDoses,0.331932,0.416602


字段fitness的值是枚举值，需要将其转换为数字，采用序号编码方式进行转换

In [45]:
df['fitness_transfer'] = pd.Categorical(df['fitness']).codes
df.drop(['fitness'], axis=1, inplace=True)
df[0:1]

,iceCream,miles_scaled,gameUsed_scaled,fitness_transfer
0,0.953952,0.331932,0.416602,1


定义一个KNN分类函数，参数分别为特征属性集合和标签集合，返回训练好的KNN模型

In [46]:
# KNN Classifier  
def knn_classifier(train_x, train_y):  
    from sklearn.neighbors import KNeighborsClassifier  
    model = KNeighborsClassifier()  
    model.fit(train_x, train_y)  
    return model

需要将原来的数据集合分割为训练集和测试集，按8:2进行切分，80%为训练集，20%为测试集

同时还需要将数据集合按列分割为属性集合和标签集合

In [49]:
from sklearn import cross_validation

# 训练集和测试集切分
train, test = cross_validation.train_test_split(df, test_size=0.2)

# 属性集合和标签集合切分
train_x = train.as_matrix()[:,:-1]
train_y = train.as_matrix()[:,-1]

test_x = test.as_matrix()[:,:-1]
test_y = test.as_matrix()[:,-1]

主程序代码，首先根据训练集得到KNN模型，然后利用该模型对测试集进行预测，并计算预测结果的准确率。

In [50]:
import numpy as np
import time
from sklearn import metrics

num_train, num_feat = train_x.shape  
num_test, num_feat = test_x.shape

# 判断标签是否为0-1标签
is_binary_class = (len(np.unique(train_y)) == 2)

print '******************** Data Info *********************'  
print '#training data: %d, #testing_data: %d, dimension: %d' % (num_train, num_test, num_feat)  
      
start_time = time.time()
model = knn_classifier(train_x, train_y)  
print 'training took %fs!' % (time.time() - start_time)  

predict = model.predict(test_x)  

# 对于0-1标签，可以计算其精确率和召回率
if is_binary_class:  
    precision = metrics.precision_score(test_y, predict)  
    recall = metrics.recall_score(test_y, predict)  
    print 'precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall)
    
# 计算准确率
accuracy = metrics.accuracy_score(test_y, predict)  
print 'accuracy: %.2f%%' % (100 * accuracy)

******************** Data Info *********************
#training data: 800, #testing_data: 200, dimension: 3
training took 0.001061s!
accuracy: 96.00%
